In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import rc
import time
rc('text', usetex=True)
rc('font', family='serif')
% matplotlib inline

In [ ]:
for i in range(29):
    print round(bins[i],3), "/", round(bins[i+1],3), "/", i

In [ ]:
b = bins[-1]
a = bins[0]
array = (b - a) * np.random.random_sample(10) + a
array[0] = 10.0

In [ ]:
index = (((array - a)/(b-a)) * 29).astype('int')

In [ ]:
print array
print index

In [4]:
(15-10.)/30

0.16666666666666666

In [3]:
%%time
rhos     = [3.5, 3.0, 2.5]
dc       = [1, 2, 3]                # Density cuts
#models = ['G13']
#type_cut = ['Stellar Mass/']
models   = ['L16']
type_cut = ['Stellar Mass/', 'SFR/']

NBIN, lmax, lmin = 30, 15, 10.
bin_width = ( lmax - lmin ) / NBIN

for model in models:
    
    ab = model
    model = ab + "_millennium"
    path = "/home/esteban/Escritorio/Practica/Data/" + model
    os.chdir( path )
    
    # Loading
    x = np.load( 'x.npy' )
    y = np.load( 'y.npy' )
    z = np.load( 'z.npy' )
    if ab == 'G13': halomass = np.log10( np.load( 'centralmvir.npy' ) + 1e-10 ) + 10
    else: halomass = np.log10( np.load( 'mdhalo.npy' ) + 1e-10 ) + 10
    type_gal = np.load( 'type.npy' )

    idx = np.where( type_gal == 0 )[0] # Positions of the centrals

    # Centrals Data
    x_cen = x[idx]
    y_cen = y[idx]
    z_cen = z[idx]
    halomass_cen = halomass[idx]
    
    del x, y, z, halomass
    
    for cut in type_cut:
        
        hod_path = path + '/hod_data/' + cut
        cat_path = path + '/Catalogues/' + cut
        os.chdir( cat_path )
        
        for k, rho in enumerate(rhos):
    
            # Loading HOD data
            #hod_path = path + '/hod_data/' + cut
            bins, HOD, Cent, Sats = np.loadtxt( hod_path + 'hod_%.1f' %rho, unpack = True )
            bins = bins - bin_width * 0.5 # Move to original position

            # Straight parameters equation
            params = np.zeros((NBIN - 1, 2), dtype = np.float32)
            for i in range( NBIN-1 ):
                y1, y2 = HOD[i], HOD[i+1]
                x1, x2 = bins[i], bins[i+1]
                m = ( y2 - y1 ) / ( x2 - x1)
                n = y1 - m * x1
                params[i] = np.array([m, n])

            mask = (halomass_cen >= bins[0]) & (halomass_cen <= bins[-1])
            halomass_cen = halomass_cen[mask]
            x_cen, y_cen, z_cen = x_cen[mask], y_cen[mask], z_cen[mask]

            #%%time
            mass_index = ((( halomass_cen - bins[0] ) / ( bins[-1] - bins[0] )) * (NBIN-1)).astype('int')
            n_gal = np.zeros(len(halomass_cen))
            for i in xrange(len(halomass_cen)):
                index = mass_index[i]
                m, n = params[index]
                log_n_gal = m * halomass_cen[i] + n
                n_gal[i] = 10**(log_n_gal)   

            #%%time
            nn_gal = np.zeros( len(n_gal) )
            for i in xrange( len(n_gal) ):
                n = n_gal[i]
                dec = n % 1
                rr = np.random.random_sample()
                if rr <= dec: nn_gal[i] = int(n) + 1
                else: nn_gal[i] = int(n)    
            
            del n_gal
            #%%time
            
            j = 0
            f = open('cat_d%i_fixed.txt' %(k+1), 'w')
            
            for i in xrange( len(x_cen) ):
                n = nn_gal[i]
                if n != 0:
                    while j < n:
                        print >> f, x_cen[i], y_cen[i], z_cen[i], halomass_cen[i]
                        j += 1
                    j = 0

            f.close()        
            data = np.loadtxt('cat_d%i_fixed.txt' %(k+1))
            np.save('cat_d%i_fixed' %(k+1), data)         


CPU times: user 13min 3s, sys: 7.36 s, total: 13min 10s
Wall time: 13min 22s


In [ ]:
import numpy as np
dec = 0.1 %1
cont = 0
for i in range(1000):
    a = np.random.random_sample()
    if a < dec:
        cont += 1
        #print a

In [ ]:
"""halo_mass = np.log10(np.load('mdhalo.npy') + 1e-10) + 10
type_gal = np.load('type.npy')
x = np.load('x.npy')
y = np.load('y.npy')
z = np.load('z.npy')

logm_min, logm_max = 10,15.
NBIN = 30
bin_width = (logm_max - logm_min) / NBIN

hod_path = path + '/hod_data/'
data = np.load(hod_path + 'hod_stellarmass_1.5.npy')

bins = data[:,0] - bin_width * 0.5
HOD  = data[:,1]
Cent = data[:,2]
Sats = data[:,3]

from scipy.interpolate import interp1d

f = interp1d(bins, HOD)
xnew = np.linspace(logm_min, logm_max - bin_width -0.01, num=50)
plt.plot(bins, HOD, 'o', xnew, f(xnew), '-')
plt.legend(['data', 'linear'], loc='best')

mask = (halo_mass >= min(bins)) & (halo_mass < max(bins))
halo_mass = halo_mass[mask]
type_gal = type_gal[mask]
x,y,z = x[mask] ,y[mask], z[mask]

idx = np.where(type_gal == 0)[0]

start_time = time.time()
#Total_Len = 0
LogN_array = np.zeros(len(idx))
i = 0
display_step = len(halo_mass[idx])/10

for mass in halo_mass[idx]:
    Log_N = f(mass)
    dec = Log_N - int(Log_N)
    rand = np.random.random_sample()
    if rand <= dec: 
        Log_N += 1
    LogN_array[i] = int(Log_N)    
    #Total_Len += 10**(Log_N)
    i += 1
    if i % display_step == 0:
        print "Step: %i Time: %3fs" % (i, (time.time() - start_time))

Total_Len = np.sum(10**(LogN_array))        
data = np.zeros((Total_Len, 3), dtype=np.float32)
x_ = x[idx]
y_ = y[idx]
z_ = z[idx]    
    
i = 0
while i < Total_Len:
    x_cat = x_[i]
    y_cat = y_[i]
    z_cat = z_[i]
    
    N = 10**(LogN_array[i])
    data[i] = np.array([x_cat, y_cat, z_cat])
"""     